# **1. Consulta del dataset**
- *Selecciona un dataset público de tu interés (por ejemplo: datos abiertos gubernamentales, estadísticas deportivas, información climática, etc.).*
- *Descárgalo o consúmelo desde su API, según corresponda.*

In [ ]:
# Importante! - tener descargado el dataset.csv en la carpeta del proyecto.

import pandas as pd

try:
    df_teenagers = -1
    # Lee el archivo ".csv" y lo guarda en un dataframe.
    df_teenagers = pd.read_csv("teen_phone_addiction_dataset.csv", sep=',') 
except Exception as e:
    print(f"Ocurrió un error: {e}")
finally:
    if isinstance(df_teenagers, pd.DataFrame):
        print("Se creó el dataFrame exitosamente.")

In [ ]:
df_teenagers.head()

In [ ]:
from dotenv import load_dotenv
import os

# Lee el archivo ".env".
load_dotenv(override=True)

# db_user = os.getenv("DB_USER")
# db_pass = os.getenv("DB_PASS")
db_name = os.getenv("DB_NAME") # Posee el nombre de la BD que se generará.
db_table_teenagers = os.getenv("DB_TABLE_TEENAGERS") # Posee el nombre de la tabla que se generará en la BD SQL.

print(f"db_name = {db_name}")
print(f"db_table_teenagers = {db_table_teenagers}")

# **2. Almacenamiento en base de datos**
- *Elige el motor de base de datos que prefieras (relacional como PostgreSQL/MySQL o NoSQL como MongoDB).*
- *Diseña las estructuras necesarias (tablas o colecciones) y carga los datos.*

In [60]:
import sqlite3

try:
    # Establece la conexión con la BD SQLite (crea el archivo 'db_teenagers.sqlite' si no existe)
    conn = sqlite3.connect(f"{db_name}.sqlite")

    # Ejecuta la consulta para crear (solo si NO existe) la tabla en la BD.
    conn.execute(f"""
    CREATE TABLE IF NOT EXISTS {db_table_teenagers} (
        ID INTEGER PRIMARY KEY,
        Name TEXT,
        Age INTEGER,
        Gender TEXT,
        Location TEXT,
        School_Grade TEXT,
        Daily_Usage_Hours REAL,
        Sleep_Hours REAL,
        Academic_Performance INTEGER,
        Social_Interactions INTEGER,
        Exercise_Hours REAL,
        Anxiety_Level INTEGER,
        Depression_Level INTEGER,
        Self_Esteem INTEGER,
        Parental_Control INTEGER,
        Screen_Time_Before_Bed REAL,
        Phone_Checks_Per_Day INTEGER,
        Apps_Used_Daily INTEGER,
        Time_on_Social_Media REAL,
        Time_on_Gaming REAL,
        Time_on_Education REAL,
        Phone_Usage_Purpose TEXT,
        Family_Communication INTEGER,
        Weekend_Usage_Hours REAL,
        Addiction_Level REAL
    )
    """)

    # Intenta crear la tabla en la BD, e intenta insertar los datos del dataframe en la tabla de la BD.
    df_teenagers.to_sql(db_table_teenagers, conn, if_exists="append", index=False)

    # Se guardan los cambios generados en la BD.
    conn.commit()
except Exception as e:
    # Si ocurre algo inesperado se asume que la tabla ya fue cargada.
    print(f"La tabla '{db_table_teenagers}' ya existe en la BD y fue cargada, u ocurrió otro error: {e}")
finally:
    # Cierra la conexión con la BD.
    conn.close()

# **3. Construcción del dashboard**
- *Utiliza una notebook (Jupyter, Colab o similar) para crear un dashboard que permite explorar la información almacenada en la base de datos.*
- *Puedes emplear herramientas como Plotly, Panel, Voila, Streamlit o widgets de Jupyter para la visualización.*

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

plt.figure(figsize=(8,5))
sns.boxplot(x="Gender", y="Sleep_Hours", data=df_teenagers)
plt.title("Distribución de horas de sueño por género")
plt.xlabel("Género")
plt.ylabel("Horas de sueño")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Promedio de horas de sueño por grado escolar
promedios = df_teenagers.groupby("School_Grade")["Sleep_Hours"].mean()

# Ordenar de menor a mayor promedio
promedios = promedios.sort_values(ascending=True)

plt.figure(figsize=(8,5))
plt.bar(promedios.index, promedios.values, color="lightgreen", edgecolor="black")
plt.title("Promedio de horas de sueño por grado escolar")
plt.xlabel("Grado escolar")
plt.ylabel("Horas de sueño")
plt.show()

In [ ]:
import matplotlib.pyplot as plt

# Rangos de adicción
bins = [0, 2, 4, 6, 8, 10]  # intervalos: 0-2, 2-4, 4-6, 6-8, 8-10

plt.figure(figsize=(8,5))
plt.hist(df_teenagers["Addiction_Level"], bins=bins, color="skyblue", edgecolor="black")
plt.title("Distribución del nivel de adicción (uso del teléfono)")
plt.xlabel("Nivel de adicción (uso del teléfono)")
plt.ylabel("Cantidad de adolescentes")
plt.xticks(bins)  # Para mostrar los límites de cada rango
plt.show()